In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [5]:
%%time
#train_identity
train_identity = dd.read_csv('../input/train_identity.csv')
train_identity = train_identity.compute()
train_identity = train_identity.reset_index(drop=True)
print("train_identity finished")
#test_identity
test_identity = dd.read_csv('../input/test_identity.csv')
test_identity = test_identity.compute()
test_identity = test_identity.reset_index(drop=True)
print("test_identity finished")
#train_transaction
train_transaction = dd.read_csv('../input/train_transaction.csv')
train_transaction = train_transaction.compute()
train_transaction = train_transaction.reset_index(drop=True)
print("train_transaction finished")
#test_transaction
test_transaction = dd.read_csv('../input/test_transaction.csv')
test_transaction = test_transaction.compute()
test_transaction = test_transaction.reset_index(drop=True)
print("test_transaction finished")
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(sample_submission.shape)

train_identity finished
test_identity finished
train_transaction finished
test_transaction finished
(506691, 2)
CPU times: user 1min 8s, sys: 11.2 s, total: 1min 19s
Wall time: 27.9 s


In [6]:
print(train_identity.shape,test_identity.shape,train_transaction.shape,test_transaction.shape)

(144233, 41) (141907, 41) (590540, 394) (506691, 393)


In [7]:
%%time
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

CPU times: user 7.88 s, sys: 7.14 s, total: 15 s
Wall time: 15 s


In [8]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0

In [9]:
used_columns = ['card1','card2','card3','card5','addr1','addr2']

In [10]:
train[used_columns].isnull().sum()

card1        0
card2     8933
card3     1565
card5     4259
addr1    65706
addr2    65706
dtype: int64

In [11]:
for i in tqdm(used_columns):
    train[i].fillna(999999,inplace=True)
    test[i].fillna(999999,inplace=True)    

100%|██████████| 6/6 [00:00<00:00, 316.92it/s]


In [13]:
for tuple_ in tqdm(list(combinations(used_columns,4))):
    a,b,c,d = tuple_[0],tuple_[1],tuple_[2],tuple_[3]
    train[a + '_' + b + '_' + c + '_' + d] = train[a].apply(str) + '_' + train[b].apply(str) + '_'  + train[c].apply(str) + '_' + train[d].apply(str)
    test[a + '_' + b + '_' + c + '_' + d] = test[a].apply(str) + '_' + test[b].apply(str) + '_'  + test[c].apply(str) + '_' + test[d].apply(str) 

100%|██████████| 15/15 [00:25<00:00,  1.72s/it]


In [14]:
print(list(train.columns)[-15:])

['card1_card2_card3_card5', 'card1_card2_card3_addr1', 'card1_card2_card3_addr2', 'card1_card2_card5_addr1', 'card1_card2_card5_addr2', 'card1_card2_addr1_addr2', 'card1_card3_card5_addr1', 'card1_card3_card5_addr2', 'card1_card3_addr1_addr2', 'card1_card5_addr1_addr2', 'card2_card3_card5_addr1', 'card2_card3_card5_addr2', 'card2_card3_addr1_addr2', 'card2_card5_addr1_addr2', 'card3_card5_addr1_addr2']


In [15]:
merged_columns = ['card1_card2_card3_card5', 'card1_card2_card3_addr1', 'card1_card2_card3_addr2', 'card1_card2_card5_addr1', 
                  'card1_card2_card5_addr2', 'card1_card2_addr1_addr2', 'card1_card3_card5_addr1', 'card1_card3_card5_addr2',
                  'card1_card3_addr1_addr2', 'card1_card5_addr1_addr2', 'card2_card3_card5_addr1', 'card2_card3_card5_addr2', 
                  'card2_card3_addr1_addr2', 'card2_card5_addr1_addr2', 'card3_card5_addr1_addr2']

In [32]:
for i in tqdm(merged_columns):
    print(i)
    le = LabelEncoder()
    le.fit(list(train[i]) + list(test[i]))
    train[i] = le.transform(train[i])
    test[i] = le.transform(test[i])
    

  0%|          | 0/15 [00:00<?, ?it/s]

card1_card2_card3_card5


  7%|▋         | 1/15 [00:01<00:19,  1.41s/it]

card1_card2_card3_addr1


 13%|█▎        | 2/15 [00:02<00:18,  1.44s/it]

...

 87%|████████▋ | 13/15 [00:18<00:02,  1.40s/it]

card2_card5_addr1_addr2


 93%|█████████▎| 14/15 [00:19<00:01,  1.39s/it]

card3_card5_addr1_addr2


100%|██████████| 15/15 [00:21<00:00,  1.38s/it]


In [47]:
for i in tqdm(merged_columns):
    temp = train.groupby(i)['isFraud'].apply(list)
    temp = temp.reset_index(drop=False)
    list_column = i+'_list'
    length_column = list_column + '_length'
    sum_column = list_column +'_sum'
    ratio_column = list_column + '_ratio'
    temp.columns = [i,list_column]
    temp[length_column] = temp[list_column].apply(len)
    temp[sum_column] = temp[list_column].apply(np.sum)
    temp[ratio_column] = temp[sum_column] / temp[length_column]
    temp_list = list(temp[(temp[length_column] >100) & (temp[ratio_column] == 0.0) ][i])
    train[i+'is_0'] = train[i].apply(lambda x: 1 if x in temp_list else 0)
    test[i+'is_0'] = test[i].apply(lambda x: 1 if x in temp_list else 0)
    temp_list = list(temp[(temp[length_column] >50) & (temp[ratio_column] > 0.30) ][i])
    train[i+'is_1'] = train[i].apply(lambda x: 1 if x in temp_list else 0)
    test[i+'is_1'] = test[i].apply(lambda x: 1 if x in temp_list else 0)
    del temp,temp_list,list_column,length_column,sum_column,ratio_column
    gc.collect()

100%|██████████| 15/15 [01:13<00:00,  3.80s/it]


In [48]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,card1_card2_card3_card5,card1_card2_card3_addr1,card1_card2_card3_addr2,card1_card2_card5_addr1,card1_card2_card5_addr2,card1_card2_addr1_addr2,card1_card3_card5_addr1,card1_card3_card5_addr2,card1_card3_addr1_addr2,card1_card5_addr1_addr2,card2_card3_card5_addr1,card2_card3_card5_addr2,card2_card3_addr1_addr2,card2_card5_addr1_addr2,card3_card5_addr1_addr2,card1_card2_card3_card5is_0,card1_card2_card3_card5is_1,card1_card2_card3_addr1is_0,card1_card2_card3_addr1is_1,card1_card2_card3_addr2is_0,card1_card2_card3_addr2is_1,card1_card2_card5_addr1is_0,card1_card2_card5_addr1is_1,card1_card2_card5_addr2is_0,card1_card2_card5_addr2is_1,card1_card2_addr1_addr2is_0,card1_card2_addr1_addr2is_1,card1_card3_card5_addr1is_0,card1_card3_card5_addr1is_1,card1_card3_card5_addr2is_0,card1_card3_card5_addr2is_1,card1_card3_addr1_addr2is_0,card1_card3_addr1_addr2is_1,card1_card5_addr1_addr2is_0,card1_card5_addr1_addr2is_1,card2_card3_card5_addr1is_0,card2_card3_card5_addr1is_1,card2_card3_card5_addr2is_0,card2_card3_card5_addr2is_1,card2_card3_addr1_addr2is_0,card2_card3_addr1_addr2is_1,card2_card5_addr1_addr2is_0,card2_card5_addr1_addr2is_1,card3_card5_addr1_addr2is_0,card3_card5_addr1_addr2is_1
0,2987000,0,86400,68.5,W,13926,999999.0,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0

In [56]:
train[['TransactionID'] + list(train.columns[-30:])].to_csv('../input/merged_train.csv',index=False)
test[['TransactionID'] + list(train.columns[-30:])].to_csv('../input/merged_test.csv',index=False)

In [ ]:
test[test['card1_card2_card5_addr1'] =='10112_360.0_166.0_191.0'].shape

In [ ]:
temp[(temp['isFraud_length'] >100) & (temp['isFraud_ratio'] == 0.0)].sort_values(by = 'isFraud_length',ascending=False)